# Assignment 6
### 12210357 徐婧珺
* Using Python and Numpy, write a class named LinearRegression, which implements the linear regression model. 
    * **__init__()** method: Initialize w0, w1 and input_dim
    * **loss()** method: Compute the MSE loss between the outputs and the targets 
    * **gradient()** method: Compute the gradient of the loss function with respect to learnable parameters 
    * **batch_update()** method: Iteratively solving the learnable parameters with batch update
    * **stochastic_update()** method: Iteratively solving the learnable parameters with stochastic online update
    * **preprocess_data()** method: Converting the inputs data and the targets data into the array in numpy
    * **train()** method: Training the linear regression model
    * **predict()** method: Predicting the target for a given input
    * **show()** method: Showing the LinearRegression Model's learnable parameters
* Make an example to test the linear regression model. 
    * Generate a data set (500 samples) with two-dimensional inputs and a single continuous output from the function $y = 2x_1 + 5x_2 + 3 + r$, where $r$ is a random noise with its value taken in the range $[-1,1]$
    * Use the genertated data to train the LinearRegression Model
    * Show the training results and some predictions

In [5]:
import numpy as np

class LinearRegression: # y(x) = w0 + w1*x，输入x可以是多维的，输出y是一维的
    def __init__(self, input_dim): # 初始化w0, w1, input_dim
        # 函数输入参数检查
        if input_dim % 1 != 0 or input_dim <=0 :
            print("When creating the object LinearRegression, the value of the input_dim should be an integer")
            exit()
        # 函数功能实现
        self.w0 = 1
        self.w1 = np.ones((1,input_dim))
        self.input_dim = input_dim
        
    def loss(self, outputs, targets): # 计算 MSE Loss
        # 函数输入参数检查
        if len(outputs) != len(targets):
            print("In Loss function, the length of the outputs and the length of the targets are not the same!")
            return False
        # 函数功能实现
        ls = 0
        for i in range(0,len(outputs)):
            ls = ls + pow((targets[i] - outputs[i]),2)
        return ls

    def gradient(self, input, target): # 根据一个数据点计算Loss跟模型参数之间的梯度，target为一个数，input为一个(1,input_dim)的向量
        # 函数输入参数检查
        if (len(input) != len(self.w1)):
            print("In gradient function, the size of the input does not fit the size of w1")
            return False
        if (len(target) != 1):
            print("In gradient function, the length of the target is not appropriate")
            return False
        # 函数功能实现
        grad = [0,list(np.zeros(len(self.w1)))]
        grad[0] = 2 * (target - (self.w0 + np.dot(self.w1, input)))
        grad[1] = np.multiply(2 * (target - (self.w0 + np.dot(self.w1, input))), input)
        return grad

    def batch_update(self, batch_size, learning_rate, inputs, targets): # 一个batch更新一次参数，target为一个(batch_size,1)的向量，input为一个(batch_size,input_dim)的向量
        # 函数输入参数检查
        if inputs.shape[0] != len(targets):
            print("In batch_update function, the length of the inputs is not as same as the length of the targets")
            return False
        if batch_size % 1 != 0 or batch_size <= 0 :
            print("In batch_update funciton, the value of the batch_size is incorrect")
            return False
        if len(targets) != batch_size:
            print("In batch_update function, the length of the inputs and the targets are not as same as the value of batch_size")
            return False
        if inputs.shape[1] != len(self.w1):
            print("In batch_update function, the dimension of the input does not fit the dimension of w1")
            return False
        # 函数功能实现
        grad_total = [0,list(np.zeros(len(self.w1)))]
        for k in range(0, batch_size):
            grad_tmp = self.gradient(inputs[k], targets[k])
            grad_total[0] = grad_total[0] + grad_tmp[0]
            grad_total[1] = np.add(grad_total[1], grad_tmp[1])
        self.w0 = self.w0 + learning_rate / batch_size * grad_total[0]
        self.w1 = np.add(self.w1, np.multiply(grad_total[1], learning_rate / batch_size))
        return True

    def stochastic_update(self, num, learning_rate, inputs, targets): # 随机选取一个组中的一个数据更新一次参数，target为一个(num,1)的向量，input为一个(num,input_dim)的向量
        # 函数输入参数检查
        if inputs.shape[0] != len(targets):
            print("In stochastic_update function, the length of the inputs is not as same as the length of the targets")
            return False
        if num % 1 != 0 or num <= 0 :
            print("In stochastic_update function, the value of the num is incorrect")
            return False
        if len(targets) != num:
            print("In stochastic_update function, the length of the inputs and the targets are not as same as the value of num")
            return False
        if inputs.shape[1] != len(self.w1):
            print("In stochastic_update function, the dimension of the input does not fit the dimension of w1")
            return False
        # 函数功能实现
        k = int(np.random.randint(0, num, size=1))
        grad_final = self.gradient(inputs[k], targets[k])
        self.w0 = self.w0 + learning_rate * grad_final[0] 
        self.w1 = np.add(self.w1, np.multiply(grad_final[1], learning_rate))
        return True

    def preprocess_data(self, predata): # 将数据转化numpy中的array
        # 函数功能实现
        data = np.array(predata)
        return data
    
    def predict(self, inputs): # predicting the target for a given input
        # 函数输入参数检查
        if np.array(inputs).shape[1] != self.input_dim:
            print("In predict function, the dimension of the input is not appropriate")
            return False
        # 函数功能实现
        return np.add(self.w0,np.dot(inputs,self.w1))
    
    def train(self, inputs, targets, learning_rate, num, update_type, w0, w1): 
        # num是更新一次参数所需数据量，update_type是指使用batch_update还是stochastic_update(可选0或1)，w0和w1的初始值
        # 函数输入参数检查
        if inputs.shape[1] != self.input_dim:
            print("In train function, the length of the input does not fit the input_dim")
        if inputs.shape[0] != len(targets):
            print("In train function, the length of the inputs and the length of the targets are not the same")
            return False
        if inputs.shape[1] != len(w1):
            print("In train function, the dimension of the input does not fit the dimension of w1")
            return False
        if num > len(inputs) or (num % 1) != 0 or num <= 0 :
            print("In train function, the value of num is not approprivate")
            return False
        if update_type != 0 and update_type != 1:
            print("In train function, The value of type_update can only be 0 or 1")
            return False
        # 函数功能实现
        self.w0 = w0
        self.w1 = w1
        inputs = self.preprocess_data(inputs)
        targets = self.preprocess_data(targets)
        e = 10e-5 # 结束训练的loss要求
        cnt = inputs.shape[0] // num
        rest = inputs.shape[0] % num
        # 进行batch_update的训练
        if update_type == 0:
            for epoch in range(0,300):
                for i in range(0,cnt):
                    self.batch_update(num, learning_rate, inputs[i * num : (i + 1)*num], targets[i * num : (i + 1)*num])
                # 处理余项
                if rest != 0:
                    grad_rest = [0,list(np.zeros(len(self.w1)))]
                    for j in range(0,rest):
                        grad_tmp = self.gradient(inputs[cnt * num + j], targets[cnt * num + j])
                        grad_rest[0] = grad_rest[0] + grad_tmp[0]
                        grad_rest[1] = np.add(grad_rest[1],grad_tmp[1])
                    self.w0 = self.w0 + learning_rate / rest * grad_rest[0] 
                    self.w1 = np.add(self.w1, np.multiply(grad_rest[1], learning_rate / rest))
                # 计算loss
                outputs = self.predict(inputs)
                if self.loss(outputs, targets) <= e:
                    break
            return True
        # 进行stochastic_update的训练
        elif update_type == 1:
            for epoch in range(0,300):
                for i in range(0,cnt):
                    self.stochastic_update(num, learning_rate, inputs[i * num : (i + 1)*num], targets[i * num : (i + 1)*num])
                # 处理余项
                if rest != 0:
                    grad_rest = np.zeros((1,2))
                    j = int(np.random.randint(0, rest, size=1))
                    grad_rest = self.gradient(inputs[cnt*num + j], targets[cnt*num + j]) # 可能有bug
                    self.w0 = self.w0 + learning_rate * grad_rest[0] 
                    self.w1 = np.add(self.w1, np.multiply(learning_rate,grad_rest[1]))
                # 计算loss
                outputs = self.predict(inputs)
                if self.loss(outputs, targets) <= e:
                    break
            return True
    
    def show(self): # 将训练结果输出
        print("w0 = ")
        print(self.w0)
        print("w1 = ")
        print(self.w1)
        return True

# 测试代码
if __name__ == '__main__':   
    input_dim = 2
    input_length = 500
    learning_rate = 0.01
    batch_size = 10
    w0 = 1
    w1 = [1,1]
    # 创建随机训练集，其模型为 y = 2x1 + 5x2 + 3 + noise
    rd = np.random.RandomState(888)
    inputs = np.add(np.array(rd.randint(0, 10, (input_length, input_dim))),np.array(rd.rand(input_length,input_dim)))
    noise = np.add(np.array(rd.randint(-1, 1, (input_length, 1))),np.array(rd.rand(input_length,1)))
    w1_true = [[2],[5]]
    w0_true = 3
    targets = np.add(noise, np.add(np.dot(inputs, w1_true), w0_true))
    # 模型的建立与训练
    model = LinearRegression(input_dim)
    print("Training with batch_update")
    model.train(inputs, targets, learning_rate, batch_size, 0, w0, w1)
    model.show()
    print(model.predict([[1,1],[2,2],[3,3]]))
    print("Training with stochastic_update")
    model.train(inputs, targets, learning_rate, batch_size, 1, w0, w1)
    model.show()
    print(model.predict([[1,1],[2,2],[3,3]]))

Training with batch_update
w0 = 
[3.03733141]
w1 = 
[2.00344437 5.00262084]
[10.04339662 17.04946184 24.05552705]
Training with stochastic_update
w0 = 
[3.03597077]
w1 = 
[1.90272495 4.97098746]
[ 9.90968318 16.7833956  23.65710801]


The training results show that our training is successful!